In [ ]:
# Import basic extensions
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Import tensorflow imagedatagenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from keras.layers import SpatialDropout2D

# Import toolboxes data and result processing
from scipy import interp
from itertools import cycle
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [ ]:

# Training directory 
train_success = os.path.join('LUMC/NECK_preprocessed/Train_extra/Success')
train_nosuccess = os.path.join('LUMC/NECK_preprocessed/Train_extra/NoSuccess_extra')

train_success_raw = os.path.join('LUMC/Neck_01flexie_selected/Train_raw/Success')
train_nosuccess_raw = os.path.join('LUMC/Neck_01flexie_selected/Train_raw/NoSuccess')

# Testing directory
test_success = os.path.join('LUMC/NECK_preprocessed/Test_extra/Success')
test_nosuccess = os.path.join('LUMC/NECK_preprocessed/Test_extra/NoSuccess_extra')

test_success_raw = os.path.join('LUMC/Neck_01flexie_selected/Test_raw/Success')
test_nosuccess_raw = os.path.join('LUMC/Neck_01flexie_selected/Test_raw/NoSuccess')

# Total datasets
train_total = os.path.join('LUMC/NECK_preprocessed/Train_extra/')
test_total = os.path.join('LUMC/NECK_preprocessed/Test_extra/')

In [ ]:
# Information from the data paths
print('total number of training images with label success:', len(os.listdir(train_success)))
print('total number of training images with label no success:', len(os.listdir(train_nosuccess)))

print('total maps training images:', len(os.listdir(train_total)))
print('total maps test images:', len(os.listdir(test_total)))

In [ ]:
# Input image normalization by rescaling, as 255 is maximum pixel value, the value of the image will be between 0 an 1
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow of training images
train_generator = train_datagen.flow_from_directory(
        'LUMC/NECK_preprocessed/Train_extra/',      # Input directory for the training images 
        classes = ['Success', 'NoSuccess_extra'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=10,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=True)            

# Flow of test images
validation_generator = validation_datagen.flow_from_directory(
     #  'LUMC/NECK_preprocessed/Test_extra/',        # Input directory for the test images 
       'LUMC/Neck_01flexie_selected/Test_raw/',   
        classes = ['Success', 'NoSuccess'],   # Names of the directory folders, which work as labels
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=10,                              # Because of the limited input images
        class_mode='binary',                        # Because of the binary classification tasks on the X-ray images
        shuffle=False)

In [ ]:
# In this part, the set-up of the actual convolutional neural network (CNN) model is described and programmed

model = tf.keras.models.Sequential([

# The first convolution layer
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.5),
    
# The second convolution
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.5),   
    
# The third convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.5),   
    
# The fourth convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(0.5),   

# Flatten results to feed into the dense layer
tf.keras.layers.Flatten(),

# First dense layer
tf.keras.layers.Dense(512, activation='relu'),

# Dense output neuron
tf.keras.layers.Dense(1, activation='sigmoid')])

    

In [ ]:
model.summary()
model.layers
model.layers[0].get_weights()

In [ ]:
# Loss, optimizer and metric determination
# https://www.jeremyjordan.me/nn-learning-rate/

import matplotlib.pyplot as plt
import keras.backend as K
from keras.callbacks import Callback
import numpy as np
from keras.callbacks import LearningRateScheduler

def step_decay_schedule(initial_lr=1e-2, decay_factor=0.75, step_size=10):
 
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule)

lr_sched = step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=2)


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=opt,  metrics=['accuracy'])

# Settings for the training-process of the CNN model
history = model.fit(train_generator,
      steps_per_epoch=30,  
      epochs=40, 
      callbacks=[lr_sched],
      verbose=1,
      validation_data = validation_generator,
      validation_steps=25)

In [ ]:
# Accurracy validation
model.evaluate(train_generator)
model.evaluate(validation_generator)

# Make predictions, based on the data generator flow from the input directory
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size
validation_generator.reset()
train_generator.reset()
preds = model.predict(validation_generator,
                      verbose=1)

In [ ]:
# False positive rate and true postive rate, based on the predictions
fpr, tpr, _ = roc_curve(validation_generator.classes, preds)

# Area under curve (AUC) calculation
roc_auc = auc(fpr, tpr) 

#Plotting of the ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='No Skill')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
# Performance per epoch (x-axis), expressed in accuracy and loss value (y-axis)

# Model accuracy estimation
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Prediction values
preds = model.predict(validation_generator,
                      verbose=1)
#print(preds)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

s = 0; # Surgery success
u = 0; # Unsure value
n = 0; # No surgery success

for i in range(0,len(preds)):
    if preds[i]<0.50:
      #  print("Surgery success")
        s=s+1
        
    else:
     #   print("No surgery success: Person has no benefit from surgery")
        n=n+1

In [ ]:
# Prediction value evaluation

print("There are " + str(s) + " people predicted to have surgery success")
print("There are " + str(n) + " people predicted with no surgery success")
#print("There are " + str(u) + " people predicted with unsure prediction value")

# Probability evaluation 
#print("There are " + str(a) + " prediction with low probability, highly unsure")
#print("There are " + str(b) + " prediction with medium probability")
#print("There are " + str(c) + " prediction with high probability, very sure")

In [ ]:
# Denote as an external tool: source... 
# https://towardsdatascience.com/convolutional-neural-network-feature-map-and-filter-visualization-f75012a5a49c#:~:text=%20Visualizing%20Feature%20maps%20or%20Activation%20maps%20generated,Normalize%20the%20array%20by%20rescaling%20it%20More%20
# https://www.analyticsvidhya.com/blog/2020/11/tutorial-how-to-visualize-feature-maps-directly-from-cnn-layers/

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# No success images
#img_path='LUMC/NECK_preprocessed/Test_extra/NoSuccess_extra/01_0574189_rotated_min15_resized_cropped_rotated_extra2.png'#whole cervical spine. 52_xray_resized_cropped.png doet het goed
#img_path='LUMC/NECK_preprocessed/Test_extra/NoSuccess_extra/01_0356963_rotated_min10_resized_cropped.png'#whole cervical spine. 52_xray_resized_cropped.png doet het goed
#img_path='LUMC/NECK_preprocessed/Test_extra/NoSuccess_extra/01_0769096_rotated_min15_resized_cropped.png'#whole cervical spine. 52_xray_resized_cropped.png doet het goed

# Success images
#img_path='LUMC/NECK_preprocessed/Test_extra/Success/01_1508719_rotated_min10_resized_cropped.png'
img_path='LUMC/NECK_preprocessed/Test_extra/Success/01_1646004_rotated_min10_resized_cropped.png'

#plt.imshow('LUMC/Resized_cropped/Train/Degeneration/87_xray_resized_cropped.png')

# Output= intermediate representations for all layers in the model
successive_outputs = [layer.output for layer in model.layers[1:]]

#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

#Load the input image
img = load_img(img_path, target_size=(200, 200))

# Convert the xray image to an array with dimension 200,200,3
xray   = img_to_array(img)                           
xray   = xray.reshape((1,) + xray.shape)

# Rescale by 1/255
xray /= 255.0

# Let's run input image through our vislauization network
# to obtain all intermediate representations for the image.
successive_feature_maps = visualization_model.predict(xray)

# Retrieve are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  print(feature_map.shape)
  if len(feature_map.shape) == 4:
    
    # Plot Feature maps for the conv / maxpool layers, not the fully-connected layers
   
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    # Postprocess the feature to be visually palatable
    for i in range(n_features):
      xray  = feature_map[0, :, :, i]
      xray -= xray.mean()
      xray /= xray.std ()
      xray *=  64 #Contrast of the images, the higher, the more contrast
      xray += 128 #Clarity of the images
      xray  = np.clip(xray, 0, 255).astype('uint8')
      
    # Tile each filter into a horizontal grid
      display_grid[:, i * size : (i + 1) * size] = xray
        
        
# Display the grid
    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' )

In [ ]:
#Iterate thru all the layers of the model
for layer in model.layers:
    if 'conv' in layer.name:
        weights, bias= layer.get_weights()
        print(layer.name)
      #  print(layer.weights)
        
        #normalize filter values between  0 and 1 for visualization
        f_min, f_max = weights.min(), weights.max()
        filters = (weights - f_min) / (f_max - f_min)  
        print(layer.name, filters.shape[3])
        filter_cnt=1
        
        #plotting all the filters
        for i in range(filters.shape[3]):
            #get the filters
            filt=filters[:,:,:, i]
            #plotting each of the channel, color image RGB channels
            for j in range(filters.shape[0]):
                ax= plt.subplot(filters.shape[3], filters.shape[0], filter_cnt  )
                ax.set_xticks([])
                ax.set_yticks([])
                plt.imshow(filt[:,:, j])
                filter_cnt+=1
        plt.show()

In [ ]:
first_layer_biases  = model.layers[0].get_weights()[1]
second_layer_weights = model.layers[1].get_weights()[0]
second_layer_biases  = model.layers[1].get_weights()[1]

conv1_weights = model.layers[0].get_weights()[0]
conv2_weights = model.layers[1].get_weights()[1]
#conv3_weights = model.layers[2].get_weights()[2]
#conv4_weights = model.layers[3].get_weights()[3]

print(conv1_weights)
print(conv2_weights)
#print(conv3_weights)

In [ ]:
success_generator = validation_datagen.flow_from_directory(
       'LUMC/NECK_preprocessed/Test_extra/',        # Input directory for the test images 
        classes=['Success'],
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=10,                              # Because of the limited input images
        shuffle=False)

In [ ]:
no_success_generator =  validation_datagen.flow_from_directory(
       'LUMC/NECK_preprocessed/Test_extra/',        # Input directory for the test images 
        classes=['NoSuccess_extra'],
        target_size=(200, 200),                     # Images from the data generator will be 200 by 200 pixels
        batch_size=10,                              # Because of the limited input images
        shuffle=False)

In [ ]:
succ_pred = model.predict(success_generator)
nosucc_pred = model.predict(no_success_generator)

succ_pred_arr = np.round(succ_pred)
#print(succ_pred_arr)
succ_pred_amount = sum(succ_pred_arr)
succ_pred_amount_not = len(succ_pred_arr) - succ_pred_amount

print('Amount that was classified as 1 in the success folder = ', int(succ_pred_amount))
print('Amount that was classified as 0 in the success folder = ', int(succ_pred_amount_not))


In [ ]:
no_succ_pred_arr = np.round(nosucc_pred)
no_succ_pred_amount = sum(no_succ_pred_arr)
no_succ_pred_amount_not = len(no_succ_pred_arr) - no_succ_pred_amount

print('Amount that was classified as 1 in the no success folder = ', int(no_succ_pred_amount))
print('Amount that was classified as 0 in the no success folder = ', int(no_succ_pred_amount_not))


In [ ]:
#Success folder is in het echt 0
#No succes folder is een 1

cf_matrix = ([[int(succ_pred_amount_not), int(succ_pred_amount)],
       [int(no_succ_pred_amount_not), int(no_succ_pred_amount)]])

import seaborn as sns
ax= plt.subplot()
sns.heatmap(cf_matrix, annot=True,fmt='.0f', cmap='Blues', ax = ax)

#Labels of the axes
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Success', 'No Success']); ax.yaxis.set_ticklabels(['Success', 'No Success']);

In [ ]:
import math
from math import *

cf_matrix

success_total = (succ_pred_amount_not)+(succ_pred_amount)
nosuccess_total = (no_succ_pred_amount)+(no_succ_pred_amount_not)

print('True positive rate TP = ', int(succ_pred_amount_not)/(success_total))
print('True negative rate TN = ', int(no_succ_pred_amount)/((nosuccess_total)))
print('False positive rate FP = ', int(no_succ_pred_amount_not)/((nosuccess_total)))
print('False negative rate FN = ', int(succ_pred_amount)/((success_total)))

TP = (succ_pred_amount_not)/(success_total)
TN = (no_succ_pred_amount)/(nosuccess_total)
FP = (no_succ_pred_amount_not)/(nosuccess_total)
FN = (succ_pred_amount)/(success_total)

print('Positive predictive value (=precision?)= ', (TP)/((TP) + (FP)))
print('Negative predictive value = ', (TN)/((FN) + (TN)))
print('Sensitivity (=recall?)= ', (TP)/((TP) + (FN)))
print('Specificity = ', (TN)/((FP) + (TN)))

print('Precision= ', (TP)/((TP) + (FP)))
print('Recall= ', (TP)/((TP) + (FN)))

precision = (TP)/((TP) + (FP))
recall = (TP)/((TP) + (FN))

F1 = ((2*(precision)*(recall))/((precision)+(recall)))
print('F1 score = ', F1)

MCC = ((TP*TN)-(FP*TN))/(sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))
print('Matthews correlation coefficient MCC = ', MCC)

accuracy = ((TP+TN)/((TP+FN+TN+FP)))
print('Overall accuracy based on CM = ', accuracy)



In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)